### Pandas 数据处理

In [1]:
import pandas as pd
import numpy as np

In [43]:
ti_suning = pd.read_csv('',encoding='utf-8')
ti_suning = pd.read_csv('',encoding='utf-8')
mapping = pd.read_excel('store_url_mapping.xlsx')


In [44]:
ti_suning.rename(columns={'retailer_product_code':'rpc','harvest_product_description':'product_description'}, inplace=True)

mapping['rpc'] = mapping['rpc'].astype(str)
ti_suning['rpc'] = ti_suning['rpc'].astype(str)

In [45]:
ti_suning['Store'] = 'Suning'

In [46]:
ti_mapped = ti_suning.loc[ti_suning['rpc'].isin(mapping['rpc']) * ti_suning['Store'].isin(mapping['store'])]
ti_mapped_merge = pd.merge(ti_mapped, mapping, on = ['rpc'])
ti_mapped_merge.shape

C:\Program Files\Anaconda\lib\site-packages\pandas\core\computation\expressions.py:183: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  unsupported[op_str]))


(5670, 29)

In [47]:
# Get product stage 
def get_stage(title):
    return title[title.find("段")-1:title.find("段")+1]

ti_mapped_merge['stage'] = ti_mapped_merge['product_description'].map(get_stage)
ti_mapped_merge['stage']
ti_mapped_merge.columns

Index(['found_helpful', '#comments', 'harvest_brand', 'product_description',
       'publish_time', 'rpc', 'review_date', 'review_rating', 'review_text',
       'review_title', 'top_reviewer_flag', 'upc', 'url_x',
       'verified_purchase', 'vine_voice', 'manufacturer_response',
       'manufacturer_response_text', 'parent_review', 'syndicated_from',
       'direct_review_url', 'purchase_time', 'user_level', 'variants',
       'image_url', 'Store', 'brand', 'store', 'substore', 'url_y', 'stage'],
      dtype='object')

In [48]:
# fix text and user names
ti_mapped_merge['review_text'] = ti_mapped_merge.apply(fix_text,axis=1)
#ti_mapped_merge['username'] = ti_mapped_merge.apply(fix_name,axis=1)
ti_mapped_merge['username'] = np.NaN
ti_mapped_merge['variant_1'] = np.NaN
ti_mapped_merge['variant_2'] = np.NaN
ti_mapped_merge['review_id'] = np.NaN

In [49]:
clm = ['store', 'substore', 'brand','stage', 'url_x', 'rpc', 'product_description',
       'review_id', 'review_date', 'username', 'review_text', 'review_rating',
       'variant_1', 'variant_2','image_url']
ti_mapped_merge = ti_mapped_merge[clm]
ti_mapped_merge.shape

(5670, 15)

In [50]:
#删除其中的'系统默认好评', '此用户未填写评价内容'
ti_mapped_merge= ti_mapped_merge[True - ti_mapped_merge['review_text'].str.contains('系统默认好评')]
ti_mapped_merge= ti_mapped_merge[True - ti_mapped_merge['review_text'].str.contains('此用户未填写评价内容')]
ti_mapped_merge.shape

C:\Program Files\Anaconda\lib\site-packages\pandas\core\computation\expressions.py:183: UserWarning: evaluating in Python space because the '-' operator is not supported by numexpr for the bool dtype, use '^' instead
  unsupported[op_str]))


(3247, 15)

In [51]:
ti_mapped_merge.rename(columns={'url_x':'url'}, inplace=True)
ti_mapped_merge.columns

Index(['store', 'substore', 'brand', 'stage', 'url', 'rpc',
       'product_description', 'review_id', 'review_date', 'username',
       'review_text', 'review_rating', 'variant_1', 'variant_2', 'image_url'],
      dtype='object')

In [52]:
ti_mapped_merge['brand'].value_counts()

Abbott菁智有机奶粉    3040
Wyeth启赋有机奶粉      207
Name: brand, dtype: int64

In [53]:
# save as csv file and go to SSA
ti_mapped_merge.to_csv('Nestle_Organic_reviewtext_Suning_F170601T180331_toSSA.csv', encoding='utf-8', sep=',', index=False)

### Break Here

In [9]:
local = pd.read_csv('Nestle_Organic.reviews.F20170601T20180331.2018_0402_1340.xlsx.csv', encoding= 'utf-8')
#ti = pd.read_csv('ti_nestle_Jan.csv',encoding= 'utf-8')
mapping = pd.read_excel('Organic_product_ulrs.xlsx')

# ti['username'] = np.NAN
# ti['variant_1'] = np.NAN
# ti['variant_2'] = np.NAN
# ti['review_id'] = np.NAN

local.rename(columns={'store':'Store','title':'product_description', 'creation_date':'review_date'},inplace=True)
#ti.rename(columns={'store':'Store','retailer_product_code':'rpc','harvest_product_description':'product_description'}, inplace=True)

mapping['rpc'] = mapping['rpc'].astype(str)
local['rpc'] = local['rpc'].astype(str)
#ti['rpc'] = ti['rpc'].astype(str)

mapping['store'].value_counts()

JD        45
Tmall     32
Suning     7
Name: store, dtype: int64

In [3]:
mapping['store'].value_counts()

JD        45
Tmall     32
Suning     7
Name: store, dtype: int64

In [5]:
local_mapped = local.loc[local['rpc'].isin(mapping['rpc']) * local['Store'].isin(mapping['store'])]
#ti_mapped = ti.loc[ti['rpc'].isin(mapping['rpc']) * ti['Store'].isin(mapping['Store'])]
#print(local_mapped.shape, ti_mapped.shape)
local_mapped.shape

C:\Program Files\Anaconda\lib\site-packages\pandas\core\computation\expressions.py:183: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  unsupported[op_str]))


(21203, 15)

In [6]:
ti_mapped['Store'].value_counts()

Suning    5838
Name: Store, dtype: int64

In [6]:
local_mapped_merge = pd.merge(local_mapped, mapping, on = ['rpc'])
#ti_mapped_merge = pd.merge(ti_mapped, mapping, on = ['rpc'])
#local_mapped_merge.shape, ti_mapped_merge.shape
local_mapped_merge.shape

(21203, 19)

In [7]:
local_mapped_merge.columns

Index(['Store', 'sub_store', 'url_x', 'rpc', 'product_description',
       'review_id', 'review_date', 'username', 'review_text', 'review_rating',
       'variant_1', 'variant_2', 'device', 'job_id', 'download_time', 'brand',
       'store', 'substore', 'url_y'],
      dtype='object')

In [17]:
def get_stage(title):
    return title[title.find("段")-1:title.find("段")+1]

local_mapped_merge_out['stage'] = local_mapped_merge_out['product_description'].map(get_stage)
local_mapped_merge_out['stage']

C:\Program Files\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


0        3段
1        3段
2        3段
3        3段
4        3段
5        3段
6        3段
7        3段
8        3段
9        3段
10       3段
11       3段
12       3段
13       3段
14       3段
15       3段
16       3段
17       3段
18       3段
19       3段
20       3段
21       3段
22       3段
23       3段
24       3段
25       3段
26       3段
27       3段
28       3段
29       3段
         ..
21173    1段
21174    1段
21175    1段
21176    1段
21177    1段
21178    1段
21179    1段
21180    2段
21181    2段
21182    2段
21183    2段
21184    2段
21185    2段
21186    2段
21187    2段
21188    2段
21189    2段
21190    2段
21191    2段
21192    2段
21193    2段
21194    2段
21195    2段
21196    2段
21197    2段
21198    2段
21199    2段
21200    2段
21201    2段
21202    2段
Name: stage, Length: 21203, dtype: object

In [18]:
clm = ['store', 'substore', 'brand','stage', 'url_x', 'rpc', 'product_description',
       'review_id', 'review_date', 'username', 'review_text', 'review_rating',
       'variant_1', 'variant_2']
clm

['store',
 'substore',
 'brand',
 'stage',
 'url_x',
 'rpc',
 'product_description',
 'review_id',
 'review_date',
 'username',
 'review_text',
 'review_rating',
 'variant_1',
 'variant_2']

In [31]:
#local_mapped_merge_out = local_mapped_merge_out[clm]
#local_mapped_merge_out.rename(columns={'url_x':'url'}, inplace=True)
local_mapped_merge_out.shape

(21203, 14)

In [9]:
local_mapped_merge_out = local_mapped_merge[clm]
#ti_mapped_merge_out = ti_mapped_merge[clm]

In [49]:
ti_mapped_merge_out.columns == local_mapped_merge_out.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True], dtype=bool)

In [50]:
nestle_jan_raw_review = pd.concat([ti_mapped_merge_out, local_mapped_merge_out])
nestle_jan_raw_review.shape

(19593, 15)

In [15]:
def fix_text(row):
    review_text = str(row["review_text"]).strip()
    review_text = review_text.replace("\\", "")
    review_text = review_text.replace("\n", "")
    review_text = review_text.replace("\t", "")
    review_text = review_text.replace("\r", "")
    review_text = review_text.replace("\\", "")
    review_text = review_text.replace("\"", "")
    review_text = review_text.replace("\'", "")
    review_text = review_text.replace(",", "，")
    review_text = review_text.replace("*", "")
    review_text = review_text.replace("&gt;", "")
    review_text = review_text.replace("&lt;", "")
    review_text = review_text.replace("&hellip;", "")
    review_text = review_text.replace("&ldquo;", "")
    review_text = review_text.replace("&rdquo;", "")
    review_text = review_text.replace("&nbsp;", "")
    review_text = review_text.replace("&sub;", "")
    review_text = review_text.replace("&omega;", "")
    review_text = review_text.replace("&acute;", "")
    review_text = review_text.replace("&cap;", "")
    review_text = review_text.replace("&Sigma;", "")
    review_text = review_text.replace("&yuml;", "")
    review_text = review_text.replace("&zwnj;", "")
    review_text = review_text.replace(";", "；")
    return review_text

In [16]:
def fix_name(row):
    review_text = str(row["username"]).strip()
    review_text = review_text.replace("\\", "")
    review_text = review_text.replace("\n", "")
    review_text = review_text.replace("\t", "")
    review_text = review_text.replace("\r", "")
    review_text = review_text.replace("\\", "")
    review_text = review_text.replace("\"", "")
    review_text = review_text.replace("\'", "")
    review_text = review_text.replace(",", "，")
    review_text = review_text.replace("*", "")
    review_text = review_text.replace("&gt;", "")
    review_text = review_text.replace("&lt;", "")
    review_text = review_text.replace("&hellip;", "")
    review_text = review_text.replace("&ldquo;", "")
    review_text = review_text.replace("&rdquo;", "")
    review_text = review_text.replace("&nbsp;", "")
    review_text = review_text.replace("&sub;", "")
    review_text = review_text.replace("&omega;", "")
    review_text = review_text.replace("&acute;", "")
    review_text = review_text.replace("&cap;", "")
    review_text = review_text.replace("&Sigma;", "")
    review_text = review_text.replace("&yuml;", "")
    review_text = review_text.replace("&zwnj;", "")
    review_text = review_text.replace(";", "；")
    return review_text

In [34]:
local_mapped_merge_out['review_text'] = local_mapped_merge_out.apply(fix_text,axis=1)
local_mapped_merge_out['username'] = local_mapped_merge_out.apply(fix_name,axis=1)

In [37]:
local_mapped_merge_out= local_mapped_merge_out[True - local_mapped_merge_out['review_text'].str.contains('系统默认好评')]
local_mapped_merge_out= local_mapped_merge_out[True - local_mapped_merge_out['review_text'].str.contains('此用户未填写评价内容')]
local_mapped_merge_out= local_mapped_merge_out[True - local_mapped_merge_out['review_text'].str.contains('此用户没有')]
local_mapped_merge_out= local_mapped_merge_out[True - local_mapped_merge_out['review_text'].str.contains('此用户未及时')]
local_mapped_merge_out.shape

C:\Program Files\Anaconda\lib\site-packages\pandas\core\computation\expressions.py:183: UserWarning: evaluating in Python space because the '-' operator is not supported by numexpr for the bool dtype, use '^' instead
  unsupported[op_str]))


(20736, 14)

In [43]:
?local.pivot_table

In [38]:
local_mapped_merge_out.to_csv("Nestle_Organic.reviews.F20170601T20180331.ToSSA.csv", index=False, sep=',', encoding='utf-8')

In [52]:
nestle_jan_raw_review['review_text'] = nestle_jan_raw_review.apply(fix_text,axis=1)

In [53]:
nestle_jan_raw_review.columns

Index(['Store_x', 'Store_y', 'Sub_store', 'product_description', 'Brand',
       'rpc', 'review_id', 'username', 'review_rating', 'variant_1',
       'variant_2', 'url_x', 'url_y', 'review_date', 'review_text'],
      dtype='object')

In [54]:
nestle_jan_raw_review.shape

(19593, 15)

In [55]:
#删除其中的'系统默认好评', '此用户未填写评价内容'
nestle_jan_raw_review= nestle_jan_raw_review[True - nestle_jan_raw_review['review_text'].str.contains('系统默认好评')]
nestle_jan_raw_review= nestle_jan_raw_review[True - nestle_jan_raw_review['review_text'].str.contains('此用户未填写评价内容')]
nestle_jan_raw_review.shape

C:\Program Files\Anaconda\lib\site-packages\pandas\core\computation\expressions.py:183: UserWarning: evaluating in Python space because the '-' operator is not supported by numexpr for the bool dtype, use '^' instead
  unsupported[op_str]))


(17891, 15)

In [56]:
del nestle_jan_raw_review['Store_y']
del nestle_jan_raw_review['url_y']
nestle_jan_raw_review.rename(columns={'Store_x':'Store','url_x':'url'}, inplace=True)

nestle_jan_raw_review.columns
nestle_jan_raw_review['Store'].value_counts()

JD        7934
Tmall     5630
Suning    4327
Name: Store, dtype: int64

In [17]:
nestle_feb_1 = nestle_jan_raw_review[nestle_jan_raw_review['review_date'].str.contains('2018-02-')]
# nestle_feb_1.to_csv('nestle_feb_1.csv', index= False, encoding= 'utf-8')

In [18]:
nestle_jan_raw_review = nestle_jan_raw_review[True - nestle_jan_raw_review['review_date'].str.contains('2018-02-')]
nestle_jan_raw_review['Brand'].value_counts()

C:\Program Files\Anaconda\lib\site-packages\pandas\core\computation\expressions.py:183: UserWarning: evaluating in Python space because the '-' operator is not supported by numexpr for the bool dtype, use '^' instead
  unsupported[op_str]))


Gerber    8854
NAN       5745
NAN HA    2082
MAM         28
Name: Brand, dtype: int64

In [19]:
# 写出文件(已经将local和ti两部分数据源合并)

nestle_jan_raw_review.to_csv('nestle_jan_raw_review.csv', index=False, encoding='utf-8')

### Review of Feb.


In [57]:
local_feb = pd.read_csv('local_nestle_feb.xlsx.csv', encoding='utf-8')
ti_feb = pd.read_csv('ti_nestle_feb.csv', encoding= 'utf -8')

In [58]:
local_feb.shape, ti_feb.shape

((33449, 15), (23113, 13))

In [25]:
# nestle_feb_1.shape

In [59]:

ti_feb['username'] = np.NAN
ti_feb['variant_1'] = np.NAN
ti_feb['variant_2'] = np.NAN
ti_feb['review_id'] = np.NAN

local_feb.rename(columns={'store':'Store','retailer_product_code':'rpc','title':'product_description', 'creation_date':'review_date'},inplace=True)
ti_feb.rename(columns={'store':'Store','retailer_product_code':'rpc','harvest_product_description':'product_description'}, inplace=True)

mapping['rpc'] = mapping['rpc'].astype(str)
local_feb['rpc'] = local_feb['rpc'].astype(str)
ti_feb['rpc'] = ti_feb['rpc'].astype(str)


In [60]:
local_feb_mapped = local_feb.loc[local_feb['rpc'].isin(mapping['rpc'])]
ti_feb_mapped = ti_feb.loc[ti_feb['rpc'].isin(mapping['rpc']) * ti_feb['Store'].isin(mapping['Store'])]
local_feb_mapped.shape, ti_feb_mapped.shape

C:\Program Files\Anaconda\lib\site-packages\pandas\core\computation\expressions.py:183: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  unsupported[op_str]))


((19410, 15), (2364, 17))

In [61]:
local_feb_mapped_merge = pd.merge(local_feb_mapped, mapping, on = ['rpc'])
ti_feb_mapped_merge = pd.merge(ti_feb_mapped, mapping, on = ['rpc'])
local_feb_mapped_merge.shape, ti_feb_mapped_merge.shape


((19410, 20), (2364, 22))

In [62]:
local_feb_mapped_merge.columns

Index(['Store_x', 'sub_store', 'url_x', 'rpc', 'product_description',
       'review_id', 'review_date', 'username', 'review_text', 'review_rating',
       'variant_1', 'variant_2', 'device', 'job_id', 'download_time',
       'Store_y', 'Sub_store', 'Store_url', 'Brand', 'url_y'],
      dtype='object')

In [63]:
local_feb_mapped_merge_out = local_feb_mapped_merge[clm]
ti_feb_mapped_merge_out = ti_feb_mapped_merge[clm]
local_feb_mapped_merge_out.columns

Index(['Store_x', 'Store_y', 'Sub_store', 'product_description', 'Brand',
       'rpc', 'review_id', 'username', 'review_rating', 'variant_1',
       'variant_2', 'url_x', 'url_y', 'review_date', 'review_text'],
      dtype='object')

In [28]:
nestle_feb_1.rename(columns={'Store':'Store_x', 'url':'url_x'}, inplace= True)
nestle_feb_1.columns

Index(['Store_x', 'Store_y', 'Sub_store', 'Store_url', 'product_description',
       'Brand', 'rpc', 'review_id', 'username', 'review_rating', 'variant_1',
       'variant_2', 'url_x', 'url_y', 'review_date', 'review_text'],
      dtype='object')

In [68]:
ti_feb_mapped_merge_out.shape, local_feb_mapped_merge_out.shape,nestle_jan_raw_review.shape

((2364, 13), (19410, 13), (17891, 13))

In [67]:
del ti_feb_mapped_merge_out['Store_y']
del ti_feb_mapped_merge_out['url_y']

del local_feb_mapped_merge_out['Store_y']
del local_feb_mapped_merge_out['url_y']

In [70]:
ti_feb_mapped_merge_out.rename(columns={'Store_x':'Store','url_x':'url'}, inplace=True)
local_feb_mapped_merge_out.rename(columns={'Store_x':'Store','url_x':'url'}, inplace=True)

C:\Program Files\Anaconda\lib\site-packages\pandas\core\frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [72]:
ti_feb_mapped_merge_out.columns == nestle_jan_raw_review.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True], dtype=bool)

In [108]:
nestle_2m_raw_review = pd.concat([ti_feb_mapped_merge_out, local_feb_mapped_merge_out,nestle_jan_raw_review])
nestle_2m_raw_review.shape

(39665, 13)

In [109]:
nestle_2m_raw_review['review_text'] = nestle_2m_raw_review.apply(fix_text,axis=1)
nestle_2m_raw_review.shape

(39665, 13)

In [110]:
#删除其中的'系统默认好评', '此用户未填写评价内容'
nestle_2m_raw_review= nestle_2m_raw_review[True - nestle_2m_raw_review['review_text'].str.contains('系统默认好评')]
nestle_2m_raw_review= nestle_2m_raw_review[True - nestle_2m_raw_review['review_text'].str.contains('此用户未填写评价内容')]

#del nestle_2m_raw_review['Store_y']
#del nestle_2m_raw_review['url_y']
#nestle_2m_raw_review.rename(columns={'Store_x':'Store','url_x':'url'}, inplace=True)
nestle_2m_raw_review.columns

C:\Program Files\Anaconda\lib\site-packages\pandas\core\computation\expressions.py:183: UserWarning: evaluating in Python space because the '-' operator is not supported by numexpr for the bool dtype, use '^' instead
  unsupported[op_str]))


Index(['Store', 'Sub_store', 'product_description', 'Brand', 'rpc',
       'review_id', 'username', 'review_rating', 'variant_1', 'variant_2',
       'url', 'review_date', 'review_text'],
      dtype='object')

In [111]:
nestle_2m_raw_review.head(4)

,Store,Sub_store,product_description,Brand,rpc,review_id,username,review_rating,variant_1,variant_2,url,review_date,review_text
0,Suning,雀巢母婴自营旗舰店,自营 雀巢 (Nestle) 能恩3 幼儿配方奶粉 3段（12-36个月）900克,NAN,610800875,NaN,NaN,5,NaN,NaN,https://product.suning.com/0000000000/61080087...,2018-02-21T09:42:38,宝宝吃了不上火，孩子的营养更均衡，雀巢奶粉大品牌值得信赖，宝贝健康成长
1,Suning,雀巢母婴自营旗舰店,自营 雀巢 (Nestle) 能恩3 幼儿配方奶粉 3段（12-36个月）900克,NAN,610800875,NaN,NaN,5,NaN,NaN,https://product.suning.com/0000000000/61080087...,2018-02-21T17:35:27,奶粉质量挺好的，很值得信赖，活动优惠力度比较大 很好喝的一款奶粉宝宝喝了不上火会继续购买、物...
2,Suning,雀巢母婴自营旗舰店,自营 雀巢 (Nestle) 能恩3 幼儿配方奶粉 3段（12-36个月）900克,NAN,610800875,NaN,NaN,5,NaN,NaN,https://product.suning.com/0000000000/61080087...,2018-02-22T02:33:08,东西收到好多天了用过了才评价发货速度快包装的很严密与卖家描述的一致很好哦
3,Suning,雀巢母婴自营旗舰店,自营 雀巢 (Nestle) 能恩3 幼儿配方奶粉 3段（12-36个月）900克,NAN,610800875,NaN,NaN,5,NaN,NaN,https://product.suning.com/0000000000/61080087...,2018-02-21T22:43:27,商品质量非常好。卖家服务态度贼好。在此表示感谢。物流也贼快！


In [112]:
nestle_2m_raw_review.columns ==nestle_MOM_raw_review.columns


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True], dtype=bool)

In [113]:
nestle_MOM_raw_review.columns

Index(['Store', 'Sub_store', 'product_description', 'Brand', 'rpc',
       'review_id', 'username', 'review_rating', 'variant_1', 'variant_2',
       'url', 'review_date', 'review_text'],
      dtype='object')

In [85]:
# del nestle_MOM_raw_review['Store_url']

In [114]:
nestle_2m_raw_review = pd.concat([nestle_2m_raw_review, nestle_MOM_raw_review])
nestle_2m_raw_review.shape

(39608, 13)

In [87]:
nestle_2m_raw_review.columns

Index(['Store', 'Sub_store', 'product_description', 'Brand', 'rpc',
       'review_id', 'username', 'review_rating', 'variant_1', 'variant_2',
       'url', 'review_date', 'review_text'],
      dtype='object')

In [115]:
nestle_2m_raw_review['Brand'].value_counts()

Gerber    21754
NAN       12220
NAN HA     4775
MAM         859
Name: Brand, dtype: int64

In [116]:
def get_stage(s):
    idx = None
    if s.find('阶段') == -1:
        idx = s.find('段')
        return s[(idx-1):(idx+1)]
    else:
        idx = s.find('阶段')
        return s[(idx-1):(idx+2)]


In [117]:
clm = nestle_2m_raw_review.columns
clm = ['Store', 'Sub_store', 'product_description', 'Brand', 'stage', 'rpc',
       'review_id', 'username', 'review_rating', 'variant_1', 'variant_2',
       'url', 'review_date', 'review_text']

In [121]:
nestle_2m_raw_review['stage'] = nestle_2m_raw_review['product_description'].apply(get_stage)
nestle_2m_raw_review = nestle_2m_raw_review[clm]
nestle_2m_raw_review.tail(3)

,Store,Sub_store,product_description,Brand,stage,rpc,review_id,username,review_rating,variant_1,variant_2,url,review_date,review_text
754,JD,雀巢官方专卖店,雀巢（NESTLE） 孕产妇奶粉 妈妈配方奶粉 （孕妇产妇适用） 900g*2罐,MAM,,13516246556,1.113196e+10,s***g,5,900g*2罐,NaN,https://item.jd.com/13516246556.html,2018-01-04 21:17:23,哇哦，特别超值！准备启用?中啦
755,JD,雀巢官方专卖店,雀巢（NESTLE） 孕产妇奶粉 妈妈配方奶粉 （孕妇产妇适用） 900g*2罐,MAM,,13516246556,1.112657e+10,j***v,5,900g*2罐,NaN,https://item.jd.com/13516246556.html,2018-01-02 22:27:27,一直都喝的这款奶粉，宝宝长得壮壮的
756,JD,雀巢官方专卖店,雀巢（NESTLE） 孕产妇奶粉 妈妈配方奶粉 （孕妇产妇适用） 900g*2罐,MAM,,13516246556,1.112335e+10,y***7,5,900g*2罐,NaN,https://item.jd.com/13516246556.html,2018-01-01 20:42:41,买了好多次，从怀孕就喝雀巢孕妇奶粉，一直都是信奈你家。


In [124]:
nestle_2m_raw_review['Brand'].value_counts()
nestle_2m_raw_review['username'] = nestle_2m_raw_review.apply(fix_name,axis=1)

In [128]:
nestle_2m_raw_review.tail(3)

,Store,Sub_store,product_description,Brand,stage,rpc,review_id,username,review_rating,variant_1,variant_2,url,review_date,review_text
754,JD,雀巢官方专卖店,雀巢（NESTLE） 孕产妇奶粉 妈妈配方奶粉 （孕妇产妇适用） 900g*2罐,MAM,,13516246556,1.113196e+10,sg,5,900g*2罐,NaN,https://item.jd.com/13516246556.html,2018-01-04 21:17:23,哇哦，特别超值！准备启用?中啦
755,JD,雀巢官方专卖店,雀巢（NESTLE） 孕产妇奶粉 妈妈配方奶粉 （孕妇产妇适用） 900g*2罐,MAM,,13516246556,1.112657e+10,jv,5,900g*2罐,NaN,https://item.jd.com/13516246556.html,2018-01-02 22:27:27,一直都喝的这款奶粉，宝宝长得壮壮的
756,JD,雀巢官方专卖店,雀巢（NESTLE） 孕产妇奶粉 妈妈配方奶粉 （孕妇产妇适用） 900g*2罐,MAM,,13516246556,1.112335e+10,y7,5,900g*2罐,NaN,https://item.jd.com/13516246556.html,2018-01-01 20:42:41,买了好多次，从怀孕就喝雀巢孕妇奶粉，一直都是信奈你家。


In [132]:
nestle_2m_raw_review.to_csv('nestle_2m_raw_review.csv', index= False, encoding='utf-8')


In [129]:
nestle_2m_raw_review['Brand'].value_counts()

Gerber    21754
NAN       12220
NAN HA     4775
MAM         859
Name: Brand, dtype: int64

In [130]:
nestle_2m_raw_review_dedup = nestle_2m_raw_review.drop_duplicates()

(39608, 14)

In [131]:
nestle_2m_raw_review_dedup = nestle_2m_raw_review.drop_duplicates()
nestle_2m_raw_review_dedup.shape

(38354, 14)

In [133]:
nestle_2m_raw_review_dedup.to_csv('nestle_2m_raw_review_dedup.csv', index= False, encoding='utf-8')

### MOM Jan. and Feb.

In [39]:
local_MOM = pd.read_csv('local_MOM_nestle_jan_feb.xlsx.csv', encoding= 'utf-8')
ti_MOM = pd.read_csv('ti_MOM__jan_feb.csv', encoding= 'utf-8')

local_MOM.shape, ti_MOM.shape

((769, 15), (44, 23))

In [40]:
def get_store(url):
    if 'amazon' in url:
        return 'Amazon'
    elif 'suning' in url:
        return 'Suning'
    elif 'yhd' in url:
        return 'YHD'
    else:
        return None

ti_MOM['store'] = ti_MOM['url'].map(get_store)
ti_MOM['store'].value_counts()

Suning    44
Name: store, dtype: int64

In [41]:
ti_MOM['username'] = np.NAN
ti_MOM['variant_1'] = np.NAN
ti_MOM['variant_2'] = np.NAN
ti_MOM['review_id'] = np.NAN

# 提前修改列名
local_MOM.rename(columns={'store':'Store','retailer_product_code':'rpc','title':'product_description', 'creation_date':'review_date'},inplace=True)
ti_MOM.rename(columns={'store':'Store','retailer_product_code':'rpc','harvest_product_description':'product_description'}, inplace=True)

# 修改rpc列的数据格式, 为后面匹配做准备
mapping['rpc'] = mapping['rpc'].astype(str)
local_MOM['rpc'] = local_MOM['rpc'].astype(str)
ti_MOM['rpc'] = ti_MOM['rpc'].astype(str)

# 选取所需要的url (根据rpc, ti 的还要考虑store)
local_MOM_mapped = local_MOM.loc[local_MOM['rpc'].isin(mapping['rpc'])]
ti_MOM_mapped = ti_MOM.loc[ti_MOM['rpc'].isin(mapping['rpc']) * ti_MOM['Store'].isin(mapping['Store'])]

# 和mapping 文件做条件合并 
local_MOM_mapped_merge = pd.merge(local_MOM_mapped, mapping, on = ['rpc'])
ti_MOM_mapped_merge = pd.merge(ti_MOM_mapped, mapping, on = ['rpc'])


# 选取所需列
local_MOM_mapped_merge_out = local_MOM_mapped_merge[clm]
ti_MOM_mapped_merge_out = ti_MOM_mapped_merge[clm]

# 合并ti 和 local 
nestle_MOM_raw_review = pd.concat([ti_MOM_mapped_merge_out, local_MOM_mapped_merge_out])

nestle_MOM_raw_review['review_text'] = nestle_MOM_raw_review.apply(fix_text,axis=1)
nestle_MOM_raw_review.shape

#删除其中的'系统默认好评' '此用户未填写评价内容'
nestle_MOM_raw_review= nestle_MOM_raw_review[True - nestle_MOM_raw_review['review_text'].str.contains('系统默认好评')]
nestle_MOM_raw_review= nestle_MOM_raw_review[True - nestle_MOM_raw_review['review_text'].str.contains('此用户未填写评价内容')]
# 删除合并文件后重复的列
del nestle_MOM_raw_review['Store_y']
del nestle_MOM_raw_review['url_y']

# 修改列名, 完成所有MOM产品的合并, 清洗
nestle_MOM_raw_review.rename(columns={'Store_x':'Store','url_x':'url'}, inplace=True)
nestle_MOM_raw_review.shape



C:\Program Files\Anaconda\lib\site-packages\pandas\core\computation\expressions.py:183: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  unsupported[op_str]))
C:\Program Files\Anaconda\lib\site-packages\pandas\core\computation\expressions.py:183: UserWarning: evaluating in Python space because the '-' operator is not supported by numexpr for the bool dtype, use '^' instead
  unsupported[op_str]))


(798, 14)

In [577]:
# 按照January 和February 进行数据拆分
nestle_MOM_jan_raw_review = nestle_MOM_raw_review[nestle_MOM_raw_review['review_date'].str.contains('2018-01')]
nestle_MOM_feb_raw_review = nestle_MOM_raw_review[nestle_MOM_raw_review['review_date'].str.contains('2018-02')]

In [578]:
nestle_MOM_jan_raw_review.shape, nestle_MOM_feb_raw_review.shape

((336, 14), (465, 14))

In [583]:
# 将February的所有数据合并, 下一步运POSTag java 程序
nestle_feb_raw_review_final = pd.concat([nestle_feb_raw_review, nestle_MOM_feb_raw_review])

# 写出文件
nestle_feb_raw_review_final.to_csv('nestle_feb_raw_review_final.csv', index= False, encoding= 'utf-8')

In [582]:
nestle_feb_raw_review_final['Brand'].value_counts()

Gerber    12973
NAN        6504
NAN HA     2710
MOM         498
Name: Brand, dtype: int64

In [591]:
nestle_feb_raw_review_final.columns

Index(['Store', 'Sub_store', 'Store_url', 'product_description', 'Brand',
       'rpc', 'review_id', 'username', 'review_rating', 'variant_1',
       'variant_2', 'url', 'review_date', 'review_text'],
      dtype='object')

In [585]:
nestle_jan_raw_review_final = pd.concat([nestle_jan_raw_review, nestle_MOM_jan_raw_review])
nestle_jan_raw_review_final['Brand'].value_counts()

Gerber    8931
NAN       5815
NAN HA    2099
MOM        364
Name: Brand, dtype: int64

In [ ]:
nestle_MOM_jan_raw_review.to_csv('nestle_MOM_jan_raw_review.csv', index= False, encoding='utf-8')


In [586]:
nestle_jan_raw_review_final.to_csv('nestle_jan_raw_review_final.csv',index=False, encoding='utf-8')

In [588]:
nestle_MOM_jan_raw_review.to_csv('nestle_MOM_jan_raw_review.csv', index=False, encoding='utf-8')

In [66]:
mapping = pd.read_csv('store_urls.csv', encoding='utf-8')
mapping_final = mapping[['Sub_store', 'Store_url']]
mapping_final.drop_duplicates(inplace=True)
mapping_final.shape, mapping.shape

C:\Program Files\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


((9, 2), (364, 6))

In [63]:
final = pd.read_csv('nestle_2m_raw_review_deduped_step2.csv', sep=';', encoding='utf-8')
final.shape

C:\Program Files\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(105907, 22)

In [67]:
final['rpc'] = final['rpc'].astype(str)
mapping_final.head(3)

,Sub_store,Store_url
0,雀巢母婴京东自营旗舰店,https://mall.jd.com/index-1000003112.html
18,雀巢官方专卖店,https://mall.jd.com/index-666458.html
92,嘉宝京东自营旗舰店,https://mall.jd.com/index-1000002746.html


In [50]:
final.shape

(105907, 22)

In [68]:
out = pd.merge(final,mapping_final, on=['Sub_store'])
out.shape

(105907, 23)

In [69]:
out.columns

Index(['Store', 'Sub_store', 'product_description', 'Brand', 'stage', 'rpc',
       'review_id', 'username', 'review_rating', 'variant_1', 'variant_2',
       'url', 'review_date', 'review_text', 'TaggedWords', 'TaggedTags',
       'Topic', 'Sentiment', 'Attribute', 'SubAttribute', 'ReviewID',
       'SentiFlag', 'Store_url'],
      dtype='object')

In [71]:

new_clms = ['Store', 'Sub_store', 'Store_url', 'product_description', 'Brand', 'stage', 'rpc',
       'review_id', 'username', 'review_rating', 'variant_1', 'variant_2',
       'url', 'review_date', 'review_text', 'TaggedWords', 'TaggedTags',
       'Topic', 'Sentiment', 'Attribute', 'SubAttribute', 'ReviewID',
       'SentiFlag']
nestle_2m_raw_review_deduped_step2_mapped = out[new_clms]

In [74]:
nestle_2m_raw_review_deduped_step2_mapped.to_csv('nestle_2m_raw_review_deduped_step2_add_sub_store_url.csv', index=False, sep=';', encoding='utf-8')